# **DETECCIÓN DE HUSOS DURANTE EL SUEÑO EN EEG**

Presentado por:

* Camila Leyva Escalante
* Laura Peñaloza López
* Daniel Felipe Lasso Mora

In [1]:
# Importación de librerias

import numpy as np
import pandas as pd # procesar base de datos, .CSV  
import matplotlib.pyplot as plt
from scipy import fft #Transformada rápida de Fourier
print('Librerias cargadas...')

Librerias cargadas...


Lectura del archivo spindles.csv

In [3]:
df = pd.read_csv('spindles.csv', delimiter = ";")
df

FileNotFoundError: ignored

Crear una nueva columna con el vector de tiempo en segundos, como se ve en el apartado anterior en la columna "Segundos" se realizan muestras cada 0.005 s y acotamos la base de datos en las columnas de nuestro interés

In [ ]:
df['Tiempo (s)'] = np.arange(0,(2598*0.005),0.005)
df_final = df[["Tiempo (s)", "EEG C3-A1[uV]", "EEG O1-A1[uV]", "EEG C4-A1[uV]", "EEG O2-A1[uV]"]]
df_final

En las columnas EOG Left[uV], EEG C3-A1[uV], EEG O1-A1[uV], EEG C4-A1[uV], EEG O2-A1[uV], los valores están separados por coma, por lo que no se tomarán como float por esto convertimos las comas por puntos y definimos el tipo de datos que tiene la base

In [ ]:
cambiar = ["EEG C3-A1[uV]", "EEG O1-A1[uV]", "EEG C4-A1[uV]", "EEG O2-A1[uV]"]

for i in cambiar:
  df_final[i] = [i.replace(',','.') for i in df_final[i]]

df_final = df_final.astype('float')

df_final

Graficamos los datos en función del vector de tiempo

In [ ]:
fig, axs = plt.subplots(4, sharex=True, sharey=True) #Share -> Para la misma escala en los ejes
fig.set_size_inches(9, 12)
labels = ["EEG C3-A1[uV]", "EEG O1-A1[uV]", "EEG C4-A1[uV]", "EEG O2-A1[uV]"]
colors = ["g","r",'c',"k"]
fig.suptitle('Graficas EEG de spindles') 


for i,ax in enumerate(axs):
  axs[i].plot(df_final['Tiempo (s)'], df_final[labels[i]],color=colors[i],label=labels[i])
  axs[i].legend(loc="upper right")


plt.xlabel('Tiempo (s)')
plt.show()

In [ ]:
df2 = df_final.copy() 
df2 = df2-np.mean(df2)
labels = ["EEG C3-A1[uV]", "EEG O1-A1[uV]", "EEG C4-A1[uV]", "EEG O2-A1[uV]"]
df2 = df2[labels].apply(fft) # Transformada de Fourier de la señal

Fs = 200 # 1/0.005
t = np.linspace(0,(len(df2)-1)/Fs, len(df2)-1, endpoint = False)
            
Espec= 2.0 / len(t) * np.abs( df2[labels] [ 0:int(len(t)/2) ] )   # Espectro de la señal
Freq = np.linspace(0.0,  Fs/2 , int(len(t)/2) )


fig, axs = plt.subplots(4, sharex=False, sharey=False) 
fig.set_size_inches(9, 12)
colors = ["g","r",'c',"k"]
fig.suptitle('Espectros del EEG ') 

for i,ax in enumerate(axs):
  axs[i].plot(Freq, Espec[labels[i]], color=colors[i], label=labels[i])
  axs[i].legend(loc="upper center")
  axs[i].grid()
  #axs[i].set_xlim([12,14])

plt.xlabel('Frecuencia (Hz)')
plt.show()


# Nueva sección

Conversión de columnas a vectores

In [ ]:
C3_A1=Espec["EEG C3-A1[uV]"].to_numpy()
O1_A1=Espec["EEG O1-A1[uV]"].to_numpy()
C4_A1=Espec["EEG C4-A1[uV]"].to_numpy()
O2_A1=Espec["EEG O2-A1[uV]"].to_numpy()

El orden de las energías corresponden a los siguientes electrodos:


1.   EEG C3-A1[uV]
2.   EEG O1-A1[uV]
3.   EEG C4-A1[uV]
4.   EEG O2-A1[uV]

Energías Delta

In [ ]:
Edelta = [0,0,0,0]
for i in range(len(Freq)):
  if Freq[i] >= 0.5:
    if Freq[i] < 4:
      Edelta[0] = (Edelta[0] + C3_A1[i])/3.5
      Edelta[1] = (Edelta[1] + O1_A1[i])/3.5
      Edelta[2] = (Edelta[2] + C4_A1[i])/3.5
      Edelta[3] = (Edelta[3] + O2_A1[i])/3.5

for j in range(len(Edelta)):
    print(j+1,".",Edelta[j])

1 . 0.3930529058489629
2 . 0.22503788981871284
3 . 0.6117907187175786
4 . 0.30563838167595725


Energías theta

In [ ]:
Etheta = [0,0,0,0]
for i in range(len(Freq)):
  if Freq[i] >= 4:
    if Freq[i] < 8:
      Etheta[0] = (Etheta[0] + C3_A1[i])/4
      Etheta[1] = (Etheta[1] + O1_A1[i])/4
      Etheta[2] = (Etheta[2] + C4_A1[i])/4
      Etheta[3] = (Etheta[3] + O2_A1[i])/4

for j in range(len(Etheta)):
    print(j+1,".",Etheta[j])

1 . 0.3167999453559453
2 . 0.18137560695553248
3 . 0.31204099204274816
4 . 0.19656283725863047


Energías alfa

In [ ]:
Ealfa = [0,0,0,0]
for i in range(len(Freq)):
  if Freq[i] >= 8:
    if Freq[i] < 13:
      Ealfa[0] = (Ealfa[0] + C3_A1[i])/5
      Ealfa[1] = (Ealfa[1] + O1_A1[i])/5
      Ealfa[2] = (Ealfa[2] + C4_A1[i])/5
      Ealfa[3] = (Ealfa[3] + O2_A1[i])/5

for j in range(len(Ealfa)):
    print(j+1,".",Ealfa[j])

1 . 0.24771052672184726
2 . 0.16269420088873937
3 . 0.17815186479060557
4 . 0.19709178384302423


Energías Beta


In [ ]:
Ebeta = [0,0,0,0]
for i in range(len(Freq)):
  if Freq[i] >= 13:
    if Freq[i] < 30:
      Ebeta[0] = (Ebeta[0] + C3_A1[i])/17
      Ebeta[1] = (Ebeta[1] + O1_A1[i])/17
      Ebeta[2] = (Ebeta[2] + C4_A1[i])/17
      Ebeta[3] = (Ebeta[3] + O2_A1[i])/17

for j in range(len(Ebeta)):
    print(j+1,".",Ebeta[j])

1 . 0.009960633666967424
2 . 0.010004044073256174
3 . 0.0073564832181821655
4 . 0.009269404297241915


Energías sigma


In [ ]:
Esigma = [0,0,0,0]
for i in range(len(Freq)):
  if Freq[i] >= 12:
    if Freq[i] < 14:
      Esigma[0] = (Esigma[0] + C3_A1[i])/2
      Esigma[1] = (Esigma[1] + O1_A1[i])/2
      Esigma[2] = (Esigma[2] + C4_A1[i])/2
      Esigma[3] = (Esigma[3] + O2_A1[i])/2

for j in range(len(Esigma)):
    print(j+1,".",Esigma[j])

1 . 1.5768654746454274
2 . 0.6319629624455227
3 . 1.4926859613140673
4 . 1.3594461665500819


Al comparar las energías de los ritmos evaluados, se encontró que la mayor energía en todos los electrodos pertenece al ritmo sigma